In [ ]:
import os
import pandas as pd
import glob
import math
import emcee
import string
import corner
import numpy as np
from tqdm import tqdm
from numba import jit
import concurrent.futures
from multiprocessing import Pool
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.ticker as mticker
from scipy.integrate import odeint
import seaborn as sns
import scipy.stats as stats
import sys

# Find and sort all text files in the ./data directory
data_files = sorted(glob.glob("./data/*.txt"))

# Dictionary to store the parsed data, grouped by data type
full_data = {}

# Loop through each file in the data_files list
for file in data_files:
    # Extract the data type (treatment and cell type) from the file name
    data_type = os.path.basename(file).split('/')[-1].split('_c')[0]
    
    # Extract the mouse name (cohort identifier) from the file name
    mouse_name = 'c' + file.split('/')[-1].split('_c')[-1].split('_t')[0]
    
    # Extract the treatment days from the file name (as integers)
    t_days = np.array([int(t) for t in file.split('/')[-1].split('_c')[-1].split('.txt')[0].split('_t')[1:]])
    
    # Load the measurement data from the file (time, tumor volume)
    data = np.loadtxt(file)
    
    # Extract the specific treatment from the data type (e.g., radiation or control)
    treatment = os.path.basename(data_type).split('_')[0]
    
    # If this data type hasn't been seen before, create a new list in full_data
    if data_type not in full_data:
        full_data[data_type] = []
    
    # Append the mouse's data (including name, tumor measurements, treatment, and treatment days) to the appropriate group
    full_data[data_type].append({
        'name': mouse_name,
        'data': data,
        'treatment': treatment,
        'treatment_days': t_days
    })

all_models = []
for group in ['radiation_sensitive', 'radiation_resistant']:
    all_mice = []
    for mouse_data in full_data[group]:
        all_mice.append(mouse_data['name'])

    bic_values_per_mouse = {}
    
    for mouse in all_mice:
        files_location = f'./Output_Calibration/ll_pars_{group}_{mouse}_*.npz'
        all_chains = sorted(glob.glob(files_location))
        
        if len(all_chains) == 0:
            print(f"No .npz files found for mouse {mouse} in group {group}")
            continue

        mouse_data = next((m for m in full_data[group] if m['name'] == mouse), None)
        if mouse_data is None:
            print(f"Mouse {mouse} not found in the group {group}")
            continue

        n = len(mouse_data['data']) 

        bic_values_per_mouse[mouse] = {} 
        
        text_to_print = mouse + ' '
        model_to_print = 'Mouse '

        for chain in all_chains:
            model_name = chain.split('_')[-1].split('.')[0]
            
            npzfile = np.load(chain)
            max_ll = npzfile['max_ll']
            theta = npzfile['pars']
            k = len(theta)

            bic_value = k * np.log(n) - 2 * max_ll
            bic_values_per_mouse[mouse][model_name] = bic_value 
            text_to_print += f"& {bic_value:.2f} "
            model_to_print += f"& {model_name} "

        print(model_to_print + '\\\ \hline')
        print(text_to_print + '\\\ \hline')

    for mouse, bic_dict in bic_values_per_mouse.items():
        bic_values = list(bic_dict.values())
        bic_min = min(bic_values)

        delta_bic = [bic - bic_min for bic in bic_values]

        weights = [np.exp(-d_bic / 2) for d_bic in delta_bic]

        sum_weights = sum(weights)
        normalized_weights = [w / sum_weights for w in weights]

        weight_text = f"Weights for {mouse}: "
        for i, model in enumerate(bic_dict.keys()):
            weight_text += f"& {normalized_weights[i]:.4f} "
        
        print(weight_text + '\\\ \hline')


<>:102: SyntaxWarning: invalid escape sequence '\ '
<>:103: SyntaxWarning: invalid escape sequence '\ '
<>:125: SyntaxWarning: invalid escape sequence '\ '
<>:102: SyntaxWarning: invalid escape sequence '\ '
<>:103: SyntaxWarning: invalid escape sequence '\ '
<>:125: SyntaxWarning: invalid escape sequence '\ '
C:\Users\guilh\AppData\Local\Temp\ipykernel_25904\3870758267.py:102: SyntaxWarning: invalid escape sequence '\ '
  print(model_to_print + '\\\ \hline')
C:\Users\guilh\AppData\Local\Temp\ipykernel_25904\3870758267.py:103: SyntaxWarning: invalid escape sequence '\ '
  print(text_to_print + '\\\ \hline')
C:\Users\guilh\AppData\Local\Temp\ipykernel_25904\3870758267.py:125: SyntaxWarning: invalid escape sequence '\ '
  print(weight_text + '\\\ \hline')


Mouse & AccelCD & AccelDRT & CDwithExpFactor & CDwithLogFactor & CDwithSinFactor & CumulativeDose & DRTwithExpFactor & DRTwithLogFactor & DRTwithSinFactor & DamageRepairTime & LQ \\ \hline
c0_m16 & 336.92 & 335.42 & 294.18 & 293.54 & 280.45 & 290.34 & 297.79 & 293.68 & 282.79 & 290.58 & 340.88 \\ \hline
Mouse & AccelCD & AccelDRT & CDwithExpFactor & CDwithLogFactor & CDwithSinFactor & CumulativeDose & DRTwithExpFactor & DRTwithLogFactor & DRTwithSinFactor & DamageRepairTime & LQ \\ \hline
c0_m23 & 230.46 & 259.93 & 221.37 & 232.29 & 217.16 & 234.22 & 223.99 & 234.17 & 220.39 & 235.85 & 281.01 \\ \hline
Mouse & AccelCD & AccelDRT & CDwithExpFactor & CDwithLogFactor & CDwithSinFactor & CumulativeDose & DRTwithExpFactor & DRTwithLogFactor & DRTwithSinFactor & DamageRepairTime & LQ \\ \hline
c0_m8 & 327.03 & 328.64 & 268.26 & 278.04 & 259.53 & 280.84 & 269.81 & 280.05 & 254.05 & 282.84 & 334.59 \\ \hline
Mouse & AccelCD & AccelDRT & CDwithExpFactor & CDwithLogFactor & CDwithSinFactor & Cum